<hr>
<br>
<br>
<br>
<h1><center>Predicting Financial Markets with Machine Learning      </center></h1>
<h1><center>-      </center></h1>
<h2><center>Non-Linear Models      </center></h2>
<br>
<br>
<hr>
<br>

<br>
<br>
<h2>Purpose</h2>
<br>
<hr>
A notebook to develop an AI system aiming at trading intraday on cryptocurrencies
<br>
<br>

<br>
<br>
<h2>Imports</h2>
<br>
<hr>
<br>

In [5]:
# Pandas and Python
import pandas as pd

pd.options.display.float_format = "{:.4f}".format
import numpy as np
from tqdm import tqdm
import os
import time
from functools import partial
from multiprocessing import Pool

# Graphic Libraries
import plotly.io as pio

pio.templates.default = "simple_white"
pd.options.plotting.backend = "plotly"
import matplotlib as plt
from IPython.display import display, HTML, clear_output


# AI and stats
import statsmodels.api as sm
import xgboost
from xgboost import XGBRegressor
import torch
import sklearn


<br>
<br>
<h2>Notebook Parameters</h2>
<br>
<hr>
<br>

In [6]:
# Define data path
data_path = "data/"

# Risk free rate assumption
risk_free_rate = 0.05  # % per year
rfr_hourly = (1 + risk_free_rate) ** (1 / (24 * 365)) - 1

# Suggested training set
start_date_train = "2023-01-24"
last_date_train = "2024-01-24"

# Suggested validation set
start_date_validate = "2024-01-25"
last_date_validate = "2024-07-24"

# Test set (Unavailable)
# start_date_test = "2024-07-25"
# last_date_test = "2025-01-24"

# Maximum number of features to use
max_nb_features = 20

# Set a level of transaction costs
tc = 0.0000


<br>
<br>
<h2>Data Loading</h2>
<br>
<hr>
<br>

In [7]:
# Main data
data = pd.read_csv(
    f"{data_path}data_in_sample.csv",
    index_col=0,
    header=[0, 1],
)

# Make sure that the index is in the right format
data.index = pd.to_datetime(data.index)


In [8]:
# Load pre-processed features
features = {}
for dirpath, dirnames, filenames in os.walk(data_path):
    for filename in filenames[-max_nb_features:]:
        if "feature" in filename:
            print(f"Loading {filename}")

            # Load feature
            feature = pd.read_csv(
                f"{data_path}{filename}",
                index_col=0,
                header=[0],
            )

            # Make sure that the index is in the right format
            feature.index = pd.to_datetime(feature.index)

            # Store in the feature dict
            features[filename.replace(".csv", "")] = feature

Loading feature_914682300606.csv
Loading feature_929491324974.csv
Loading feature_936020771498.csv
Loading feature_936377278430.csv
Loading feature_938545158299.csv
Loading feature_941272397662.csv
Loading feature_950281828683.csv
Loading feature_950691475264.csv
Loading feature_952144189258.csv
Loading feature_955845031686.csv
Loading feature_961547632365.csv
Loading feature_962177675988.csv
Loading feature_962320961442.csv
Loading feature_965788928669.csv
Loading feature_977512281584.csv
Loading feature_983617492082.csv
Loading feature_986307325973.csv
Loading feature_993937281321.csv
Loading feature_996678870868.csv
Loading feature_997041357627.csv


In [9]:
list(features.keys())

['feature_914682300606',
 'feature_929491324974',
 'feature_936020771498',
 'feature_936377278430',
 'feature_938545158299',
 'feature_941272397662',
 'feature_950281828683',
 'feature_950691475264',
 'feature_952144189258',
 'feature_955845031686',
 'feature_961547632365',
 'feature_962177675988',
 'feature_962320961442',
 'feature_965788928669',
 'feature_977512281584',
 'feature_983617492082',
 'feature_986307325973',
 'feature_993937281321',
 'feature_996678870868',
 'feature_997041357627']

<br>
<br>
<h2>Analytics</h2>
<br>
<hr>
Basic Portfolio analytics to invest in some predictions of the future instruments returns
<br>
<br>


In [10]:
def expected_returns_to_positions(expected_returns):
    """
    Normalize expected returns to make it an investable portfolio

    :param expected_returns: pd.DataFrame containing expectations
                             about future instruments prices variations
    """

    # Positions will be proportional to ranked alpha
    positions = expected_returns.rank(axis=1)

    # Re-scale the leverage
    positions = positions.div(positions.abs().sum(axis=1), axis=0)

    # Make the portfolio dollar neutral
    positions = positions.sub(positions.mean(axis=1), axis=0)

    return positions


def get_sharpe(pnl_portfolio, rfr_hourly):
    """
    Compute the sharpe ratio

    :param pnl_portfolio: pd.Series of returns of the portfolio considered
    :param rfr_hourly: float, the hourly risk free rate
    """

    # Compute excess returns
    excess_returns = pnl_portfolio - rfr_hourly

    # Compute sharpe ratio
    sharpe_ratio = excess_returns.mean() / excess_returns.std() * np.sqrt(24 * 365)

    # Output
    return round(sharpe_ratio, 2)


def pnl_analytics(positions, returns, rfr_hourly, lag, tc=0):
    """
    Compute the p&l analytics of the strategy

    :param positions: pd.DataFrame, some positions that have been reached
    :param returns: pd.DataFrame containing returns of instruments
    :param rfr_hourly: float, the hourly risk free rate
    :param lag: int, the number of hours to reach the positions
    :param tc: float, the transaction costs

    """

    # Compute gross p&l
    pnl = positions.shift(1 + lag).mul(returns).sum(axis=1)

    # Compute transaction costs
    trades = positions.fillna(0).diff()
    costs = trades.abs().sum(axis=1) * tc

    # Net p&l: deduce costs from gross p&l
    pnl = pnl.sub(costs, fill_value=0)

    # Compute sharpe
    sharpe = get_sharpe(pnl, rfr_hourly)

    return {"sharpe": sharpe, "pnl": pnl}


def analyze_expected_returns(
    expected_returns,
    returns,
    rfr_hourly,
    title="a Nice Try",
    lags=[0, 1, 2, 3, 6, 12],
    tc=0,
    output_sharpe=False,
    display_results=True,
):
    """
    Provide an economic analysis of some expected_returns

    :param expected_returns: pd.DataFrame containing expectations
                             about future instruments prices variations
    """

    # Take positions as a function of expected returns
    positions = expected_returns_to_positions(expected_returns)

    # Compute p&l and sharpe for different lags
    pnl_lags = {}
    for lag in lags:
        analytics_lag = pnl_analytics(
            positions=positions, returns=returns, rfr_hourly=rfr_hourly, lag=lag, tc=tc
        )
        lag_label = f"Lag {lag}, sharpe={analytics_lag['sharpe']}"
        pnl_lags[lag_label] = analytics_lag["pnl"]

    # Display returns
    pnl_lags = pd.concat(pnl_lags, axis=1).dropna()
    if display_results:
        fig = (
            (1 + pnl_lags)
            .cumprod()
            .plot(
                title=f"Cumulative returns of {title}",
            )
        )
        fig.update_layout(yaxis_type="log")
        fig.show()

    if output_sharpe:
        for lag_label in pnl_lags.columns:
            if "Lag 0" in lag_label:
                return lag_label.split("sharpe=")[-1]


<br>
<br>
<h2>Features Standard Pre-Processing</h2>
<br>
<hr>

<br>


In [11]:
label = data["return"].loc[start_date_train:last_date_train].shift(-1).stack()

In [14]:
features_normalized = {}

for feature_name in features.keys():
    print(f"Processing {feature_name}")

    # Extract the feature
    feature_normalized = features[feature_name]

    # Rank the feature to remove outliers
    feature_normalized = feature_normalized.rank(axis=1, pct=True) - 0.5

    # Stack the feature
    feature_normalized = feature_normalized.stack().sort_index()

    # Store this normalized version
    features_normalized[feature_name] = feature_normalized

# Convert normalized features dict to a single dataframe
features_normalized = pd.concat(features_normalized, axis=1)

# Replace NaNs by average values, as OLS cannot handle NaNs effectively
features_normalized = features_normalized.fillna(0)

Processing feature_914682300606
Processing feature_929491324974
Processing feature_936020771498
Processing feature_936377278430
Processing feature_938545158299
Processing feature_941272397662
Processing feature_950281828683
Processing feature_950691475264
Processing feature_952144189258
Processing feature_955845031686
Processing feature_961547632365
Processing feature_962177675988
Processing feature_962320961442
Processing feature_965788928669
Processing feature_977512281584
Processing feature_983617492082
Processing feature_986307325973
Processing feature_993937281321
Processing feature_996678870868
Processing feature_997041357627


<br>
<br>
<h2>Non-Linear Polymodels</h2>
<br>
<hr>
By introducing non-linearities in our polymodel, we aim at capturing better the complexity of the data
<br>
<br>


<br>
<h4>Re-process features to get standardized polynomials</h4>
<br>

In [13]:
polynomials = {}

# Create a complete stacked index so that all the features afterwards
# share the same index
full_stacked_index = data["return"].fillna(0).stack().sort_index().index

for feature_name in features.keys():
    print(f"Processing {feature_name}")

    # Extract the feature
    feature_base = features[feature_name]

    # Make sure that 'event' features are only treated when active
    feature_base = feature_base.replace(0, np.nan).dropna(axis=1, how="all")

    # Rank the feature to remove outliers
    feature_base = feature_base.rank(axis=1, pct=True) - 0.5

    polynomials_feature = {}

    for degree in [1, 2, 3, 4]:
        # Elevate the feature too a given degree
        feature_normalized = feature_base**degree

        # Normalize to avoid the optimization to collapse in sklearn
        feature_normalized = feature_normalized.div(
            feature_normalized.std(axis=1), axis=0
        )

        # Stack the feature
        feature_normalized = feature_normalized.stack().sort_index()

        # Avoid NaNs values
        feature_normalized = (
            feature_normalized.fillna(0).reindex(full_stacked_index).fillna(0)
        )

        # Create hermite polynomials
        polynomials_feature[f"X{degree}"] = feature_normalized

    # Store all the polynomials together
    polynomials[feature_name] = pd.concat(polynomials_feature, axis=1)

Processing feature_914682300606
Processing feature_929491324974
Processing feature_936020771498
Processing feature_936377278430
Processing feature_938545158299
Processing feature_941272397662
Processing feature_950281828683
Processing feature_950691475264
Processing feature_952144189258
Processing feature_955845031686
Processing feature_961547632365
Processing feature_962177675988
Processing feature_962320961442


KeyboardInterrupt: 

<br>
<h4>Training polymodels</h4>
<br>



In [ ]:
# Measure time
t1 = time.time()

# Recompute the model every month, skip the first 2 months
rebalancing_dates = pd.date_range(
    start=start_date_train, end=last_date_validate, freq="ME"
)[2:]


def train_predict_period(
    last_date_train_fold,
    returns,
):
    # Define training and validation dates

    # Train the model over the last X months
    start_date_train_fold = last_date_train_fold - pd.Timedelta(days=30 * 1)

    # The model cannot be used before the first day following the training
    # (no look-forward bias)
    start_date_validate_fold = last_date_train_fold + pd.Timedelta(days=1)

    # The trained model will be used for 1 month
    last_date_validate_fold = last_date_train_fold + pd.Timedelta(days=31 * 1)

    # Log informations
    print(
        f"Train a model from date {start_date_train_fold} to date {last_date_train_fold}"
    )
    print(
        f"Predict from date {start_date_validate_fold} to date {last_date_validate_fold}"
    )
    print("")

    # Create label
    label_fold = (
        returns.loc[start_date_train_fold:last_date_train_fold].shift(-1).stack()
    )

    # Initialize predictions final dataframe
    predictions_date = pd.DataFrame(
        0,
        index=returns.index,
        columns=returns.columns,
    ).loc[start_date_validate_fold:last_date_validate_fold]

    for feature_name in polynomials.keys():
        print(feature_name)

        # Extract polynomials for the feature
        polynomials_feature = polynomials[feature_name]

        # Make sure all training data is aligned
        polynomials_feature_train_fold = polynomials_feature.loc[
            start_date_train_fold:last_date_train_fold
        ]
        common_index = label_fold.index.intersection(
            polynomials_feature_train_fold.index
        )
        polynomials_feature_train_fold = polynomials_feature_train_fold.loc[
            common_index
        ]
        label_fold_feature = label_fold.loc[common_index]

        # Get polynomials for predictions
        polynomials_feature_validate_fold = polynomials_feature.loc[
            start_date_validate_fold:last_date_validate_fold
        ]

        # Create model
        model = sklearn.linear_model.ElasticNetCV(
            l1_ratio=0.01,
            fit_intercept=True,
            cv=5,
            alphas=np.logspace(-8, -5, 50),
            n_jobs=1,
        )

        # Fit model
        model = model.fit(
            y=label_fold_feature,
            X=polynomials_feature_train_fold,
        )

        # Predict on the validation set
        predictions_feature = model.predict(polynomials_feature_validate_fold)
        predictions_feature = pd.Series(
            predictions_feature, index=polynomials_feature_validate_fold.index
        ).unstack()

        # Aggregate with other predictions
        predictions_date = predictions_date.add(predictions_feature, fill_value=0)

    print(f"date {last_date_train_fold} done")

    # Output results
    return pd.concat({str(last_date_train_fold): predictions_date}, axis=1)


# Fix all but one function parameters to iterate on the last one
partial_train_predict_period = partial(
    train_predict_period,
    returns=data["return"],
)

# Train using one core per date
with Pool(1) as pool:
    predictions = pool.map(
        partial_train_predict_period,  # function to multiprocess
        rebalancing_dates,  # values to iterate on
    )

# Reformat predictions
predictions = pd.concat(predictions, axis=1).T.groupby(level=1).sum().T


# Training finished, print time used for it
t2 = time.time()
print(f"Total Training time is {t2 - t1} seconds")

# Analyse our predictions
analyze_expected_returns(
    expected_returns=predictions.loc[start_date_validate:last_date_validate],
    returns=data["return"].loc[start_date_validate:last_date_validate],
    rfr_hourly=rfr_hourly,
    title=f"Non-Linear Polymodel, Walk-Forward Cross-Validation, Validation Set",
    lags=[0, 1, 2, 3, 6, 12],
    tc=tc,
)

<br>
<br>
<h2>XGBoost: Gradient Boosted Decision Trees</h2>
<br>
<hr>
Gradient Boosted Decision Trees are another way to introduce non-linearity in our model. This non-linearity is present in the link between the label and features, but also among the features themselves. Overfitting is limited thanks to a variety of strategies, resulting in potentially better generalization.
<br>
<br>


<br>
<h4>Defining hyper-parameters</h4>
<br>



In [17]:
# Define hyperparameters
hyperparameters = {
    "learning_rate": 0.001,
    "n_estimators": 500,
    "objective": "reg:squarederror",
    "tree_method": "hist",
    "base_score": 0,
    "max_depth": 7,
    "min_child_weight": 10,
    "subsample": 0.05,
    "colsample_bytree": 0.3,
    "min_split_loss": 0,
    "reg_lambda": 1,
    "reg_alpha": 0,
    "n_jobs": 1,
    "random_state": 0,
}

<br>
<h4>Training the models</h4>
<br>



In [18]:
# Measure time
t1 = time.time()

# Recompute the model every month, skip the first 2 months
rebalancing_dates = pd.date_range(
    start=start_date_train, end=last_date_validate, freq="ME"
)[2:]


def train_predict_period(
    last_date_train_fold,
    returns,
    hyperparameters,
):
    # Define training and validation dates

    # Train the model over the last X months
    start_date_train_fold = last_date_train_fold - pd.Timedelta(days=30 * 12)

    # The model cannot be used before the first day following the training
    # (no look-forward bias)
    start_date_validate_fold = last_date_train_fold + pd.Timedelta(days=1)

    # The trained model will be used for 1 month
    last_date_validate_fold = last_date_train_fold + pd.Timedelta(days=31 * 1)

    # Log informations
    print(
        f"Train a model from date {start_date_train_fold} to date {last_date_train_fold}"
    )
    print(
        f"Predict from date {start_date_validate_fold} to date {last_date_validate_fold}"
    )
    print("")

    # Create label
    label_fold = (
        returns.loc[start_date_train_fold:last_date_train_fold].shift(-1).stack()
    )

    # Only keep dates of the train and validation sets for the features
    features_normalized_train_fold = features_normalized.reindex(label_fold.index)
    features_normalized_validate_fold = features_normalized.sort_index().loc[
        start_date_validate_fold:last_date_validate_fold
    ]

    # Split the data along the time axis
    ts_splitter = sklearn.model_selection.TimeSeriesSplit(n_splits=5)

    # Create model
    model = XGBRegressor(**hyperparameters)

    # Fit model
    model = model.fit(
        y=label_fold,
        X=features_normalized_train_fold,
    )

    # Predict on the validation set
    predictions = model.predict(features_normalized_validate_fold)
    predictions = pd.Series(
        predictions, index=features_normalized_validate_fold.index
    ).unstack()

    # Output results
    return pd.concat({str(last_date_train_fold): predictions}, axis=1)


# Fix all but one function parameters to iterate on the last one
partial_train_predict_period = partial(
    train_predict_period,
    returns=data["return"],
    hyperparameters=hyperparameters,
)

# Train using one core per date
with Pool(16) as pool:
    predictions = pool.map(
        partial_train_predict_period,  # function to multiprocess
        rebalancing_dates,  # values to iterate on
    )

# Reformat predictions
predictions = pd.concat(predictions, axis=1).T.groupby(level=1).sum().T


# Training finished, print time used for it
t2 = time.time()
print(f"Total Training time is {t2 - t1} seconds")


: 

: 

In [ ]:
# Analyse our predictions
analyze_expected_returns(
    expected_returns=predictions.loc[start_date_validate:last_date_validate],
    returns=data["return"].loc[start_date_validate:last_date_validate],
    rfr_hourly=rfr_hourly,
    title=f"Gradient Boosted Trees, Walk-Forward Cross-Validation, Validation Set",
    lags=[0, 1, 2, 3, 6, 12],
    tc=tc,
)
